Considered features:
according to: https://github.com/sgiammy/emotion-patterns-in-music-playlists

<ul>
    <li>**Title_vector**</li>
    <li>**Lyric_vector**</li>
    <li>**%Rhymes**:<br> defined as the percentage of the number of rhymes over the number of total lines. A rhyme is defined as a rhyme between two following lines.</li>
    <li>**%Past_tense_verbs**:<br> defined as the the percentage of the number of past tense verbs over the total number of verbs.</li>
    <li>**%Present_tense_verbs**:<br>  defined as the the percentage of the number of present tense verbs over the total number of verbs.</li>
    <li>**%Future_tense_verbs**:<br>  defined as the the percentage of the number of future tense verbs over the total number of verbs, where future is just will + base form.</li>
    <li>**%ADJ**:<br> Percentage of adjectives over the total number of words.</li>
    <li>**%ADP**:<br> Percentage of adpositions (e.g. in, to, during) over the total number of words.</li>
    <li>**%ADV**:<br> Percentage of adverbs (e.g. very, tomorrow, down, where, there) over the total number of words.</li>
    <li>**%AUX**:<br> Percentage of auxiliaries (e.g. is, has (done), will (do), should (do)) over the total number of words.</li>
    <li>**%INTJ**:<br> Percentage of interjections (e.g. psst, ouch, bravo, hello) over the total number of words.</li>
    <li>**%NOUN**:<br> Percentage of nouns over the total number of words.</li>
    <li>**%NUM**:<br> Percentage of numerals over the total number of words.</li>
    <li>**%PRON**:<br> Percentage of pronouns (e.g. I, you, he, she, myself, themselves, somebody,...) over the total number of words.</li> 
    <li>**%PROPN**:<br> Percentage of proper nouns (e.g. Mary, John) over the total number of words.</li>
    <li>**%PUNCT**:<br> Percentage of puntuctuation (e.g. ., (, ), ?) over the total number of words.</li>
    <li>**%VERB**:<br> Percentage of verbs over the total number of words.</li>
    <li>**Selfish_degree**:<br> Percentage of 'I' pronouns over the total number of pronouns</li>
    <li>**%Echoism**:<br> Percentage of echoism over the total number of words, where an echoism is either a sequence of two subsequent repeated words or the repetition of a vowel in a word. </li>
    <li>**%Duplicates**:<br> Percentage of duplicate words over the total number of words</li>
    <li>**isTitleInLyric**:<br> Boolean, true if the title string is also a substring of the lyric</li>
    <li>**sentiment**:<br> Sentiment between -1 and 1</li>
    <li>**subjectivity degree**:<br> Degree of subjectivity of the text</li>
</ul>

In [2]:
import os
import json
import re
import pandas as pd

from langdetect import detect


In [59]:
def detect_language(text):
    try:
        language = detect(text)
    except Exception as e:
        print(e)
        language = None
    print(f"Detected lang = {language}")

    return language

INSTRUMENTAL_COMMENT = "This song is an instrumental"

def load_lyric_dataset(input_path):

    rows = list()
    ids = list()

    lyric_files = [os.path.join(input_path, pos_json) for pos_json in os.listdir(input_path) if pos_json.endswith('.json')]

    for file_path in lyric_files:
        with open(file_path) as f:
            song_info = json.load(f)

        try:
            id = song_info['id']
            id = id.replace("ML", "")
            id = int(id)
        except:
            id = None
            print(f"For {file_path} there is no id")

        try:
            mood = song_info['mood']
        except:
            mood = None
            print(f"For {file_path} there is no mood")

        try:
            title = song_info['title']
        except:
            title = None
            print(f"For {file_path} there is no title")

        try:
            lyric = song_info['song']['lyrics']
            if lyric == '': 
                print(f"For {file_path} lyric is empty")
        except:
            lyric = None
            print(f"For {file_path} there is no lyrics")
        
        try:
            language = song_info['song']['language']
            if language == None: language = detect_language(lyric)
        except:
            print(f"For {file_path} there is no language info in dataset")
            language = detect_language(lyric)
            

        try:
            comment = song_info['song']['//coment']
            if comment == INSTRUMENTAL_COMMENT:
                instrumental = True
                print(f"For {file_path} is instrumental\n")
            else:
                instrumental = False
        except:
            instrumental = False

        row = (mood, title, lyric, language, instrumental)
        
        rows.append(row)
        ids.append(id)

    df = pd.DataFrame(rows, columns=['mood', 'title', 'lyric', 'language', 'instrumental'], index=ids)
    
    return df

In [80]:
input_path = os.path.join('..', '..', 'database', 'lyrics')

dataset = load_lyric_dataset(input_path) 
dataset.head(10)


Detected lang = so
Detected lang = it
Detected lang = en
For ..\..\database\lyrics\ML1159.json lyric is empty
For ..\..\database\lyrics\ML1159.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1159.json is instrumental

For ..\..\database\lyrics\ML1230.json lyric is empty
For ..\..\database\lyrics\ML1230.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1230.json is instrumental

For ..\..\database\lyrics\ML1336.json lyric is empty
For ..\..\database\lyrics\ML1336.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1336.json is instrumental

For ..\..\database\lyrics\ML1349.json lyric is empty
For ..\..\database\lyrics\ML1349.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1349.json is instrumental

For ..\..\database\lyrics\ML136.json there 

,mood,title,lyric,language,instrumental
1,happy,I Want Your Sex,I Want Your Sex Lyrics[From a PSA recorded for...,en,False
10,happy,Heart of Glass,Heart of Glass Lyrics[Verse 1]\nOnce I had a l...,en,False
100,happy,Crazy Little Thing Called Love,Crazy Little Thing Called Love Lyrics[Intro]\n...,en,False
1000,happy,Almost,Almost Lyrics[Verse 1]\nI almost got drunk at ...,en,False
1001,happy,Glow,Glow Lyrics[Verse 1]\nI never thought that you...,en,False
1002,sad,The Kids,The Kids Lyrics[Verse 1]\nThey're taking her c...,en,False
1003,relaxed,Political,Political LyricsA loose grip on a thin line\nL...,en,False
1004,happy,Hold Me Tight,Hold Me Tight Lyrics[LUCY]\nIt feels so right ...,en,False
1005,relaxed,I Will Never See the Sun,I Will Never See the Sun Lyrics[Chorus]\nI wil...,en,False
1006,happy,Superfast Jellyfish,Superfast Jellyfish Lyrics[Intro]\nThis mornin...,en,False


In [81]:
dataset.describe()

,mood,title,lyric,language,instrumental
count,2000,2000,2000,1995,2000
unique,4,1978,1987,18,2
top,happy,Fire,,en,False
freq,500,3,5,1892,1995


In [76]:
def load_en_dataset(path):

    dataset = load_lyric_dataset(input_path) 

    dataset = dataset.loc[dataset['language'] == "en"]
    en_dataset = dataset.loc[dataset['instrumental'] == False]
    
    return en_dataset

In [77]:
input_path = os.path.join('..', '..', 'database', 'lyrics')

en_dataset = load_en_dataset(input_path)

Detected lang = so
Detected lang = it
Detected lang = en
For ..\..\database\lyrics\ML1159.json lyric is empty
For ..\..\database\lyrics\ML1159.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1159.json is instrumental

For ..\..\database\lyrics\ML1230.json lyric is empty
For ..\..\database\lyrics\ML1230.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1230.json is instrumental

For ..\..\database\lyrics\ML1336.json lyric is empty
For ..\..\database\lyrics\ML1336.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1336.json is instrumental

For ..\..\database\lyrics\ML1349.json lyric is empty
For ..\..\database\lyrics\ML1349.json there is no language info in dataset
No features in text.
Detected lang = None
For ..\..\database\lyrics\ML1349.json is instrumental

For ..\..\database\lyrics\ML136.json there 

In [78]:
en_dataset.describe()

,mood,title,lyric,language,instrumental
count,1892,1892,1892,1892,1892
unique,4,1873,1883,1,1
top,angry,Fire,Back to This LyricsWe were giving up time\nWe ...,en,False
freq,493,3,2,1892,1892


In [79]:
#duplicated english lyrics
df2 = en_dataset[en_dataset['lyric'].duplicated()]
df2

,mood,title,lyric,language,instrumental
186,sad,Sadness Is a Blessing,Sadness Is a Blessing Lyrics[Verse 1]\nMy woun...,en,False
1912,angry,Ghosts Of War,"Ghosts of War Lyrics[Verse 1]\nFate, silent wa...",en,False
1945,sad,Ghost In this House,Ghost in This House Lyrics[Verse 1]\nI don't p...,en,False
239,relaxed,Edge of the Ocean,Edge of the Ocean Lyrics[Verse 1]\nThere's a p...,en,False
526,angry,Sleep Now in the Fire,Sleep Now in the Fire Lyrics[Verse 1]\nYeah\nT...,en,False
659,relaxed,Faithful,Faithful Lyrics[Chorus: Sample & (Common)]\nFa...,en,False
817,relaxed,Look What You're Doin' To Me,Look What You’re Doin’ to Me LyricsI am the vo...,en,False
894,angry,Killing in the Name,TranslationsPortuguêsNederlandsKilling in the ...,en,False
992,relaxed,Back to This,Back to This LyricsWe were giving up time\nWe ...,en,False


In [82]:
#duplicated all languages lyrics
df2 = dataset[dataset['lyric'].duplicated()]
df2

,mood,title,lyric,language,instrumental
1230,relaxed,Flip Ya Lid,,None,True
1336,relaxed,Summer Madness,,None,True
1349,relaxed,Summertime,,None,True
186,sad,Sadness Is a Blessing,Sadness Is a Blessing Lyrics[Verse 1]\nMy woun...,en,False
1912,angry,Ghosts Of War,"Ghosts of War Lyrics[Verse 1]\nFate, silent wa...",en,False
1945,sad,Ghost In this House,Ghost in This House Lyrics[Verse 1]\nI don't p...,en,False
239,relaxed,Edge of the Ocean,Edge of the Ocean Lyrics[Verse 1]\nThere's a p...,en,False
379,relaxed,Reptile,,None,True
526,angry,Sleep Now in the Fire,Sleep Now in the Fire Lyrics[Verse 1]\nYeah\nT...,en,False
659,relaxed,Faithful,Faithful Lyrics[Chorus: Sample & (Common)]\nFa...,en,False


In [ ]:
temp = df.loc[df['language'] == "en"]
en_df = temp.loc[temp['instrumental'] == False]
en_df.head()

In [3]:
def clean_lyric(lyric, title):
    
    #remove title and genius annotation
    lyric = re.sub(".+Lyrics.+\]", '',  lyric)

    #removing title (exception detected)
    lyric = re.sub(f'{title}.+Lyrics', '', lyric)

    #remove exery anotation like [Verse 1], [Chorus], [Bridge], [Part 1] etc.
    lyric = re.sub('\[.+\]', '', lyric)

    #remove every ********* in the lyric
    lyric = re.sub('\*.+\*', '', lyric)

    #remove Genius anotation "You might also like"
    lyric = re.sub('You might also like', '', lyric)

    #remove Embed exist in every lyric in the end
    if lyric[-5:] == 'Embed':
        lyric = re.sub('Embed', '', lyric)
        if lyric[-1].isdigit():
            lyric = re.sub('\d', '', lyric)

    return lyric